In [ ]:
!pip install pandas transformers 
!pip install simpletransformers

In [2]:
import pandas as pd 

In [4]:
data = pd.read_csv('/content/30000_upsampling_with_original.csv')
data = pd.DataFrame(data)
data.head()

,words,labels,sentence_id
0,"""",B_SENT,0
1,จรัญ,I_SENT,0
2,"""",I_SENT,0
3,ชี้,I_SENT,0
4,ผล,I_SENT,0


In [7]:
! pip install -q datasets transformers[sentencepiece] simpletransformers

In [8]:
_CLAUSE_TAGS = ["O", "B_SENT", "I_SENT", "E_SENT"]

In [ ]:
data['words'][0]

In [18]:
len(data)

6803775

In [28]:
def convert_to_simple_transformer_format(df):
  words = []
  labels = []
  sentence_id = []
  for i in range(len(df)):
    # print(i,t)
    words.append(df['words'][i])
    labels.append(df['labels'][i])
    sentence_id.append(i)
  return pd.DataFrame(
      {"sentence_id":sentence_id, "words": words, "labels": labels}
  )
# convert_to_simple_transformer_format(data)

In [29]:
train = convert_to_simple_transformer_format(data)

In [30]:
len(train)

6803775

In [31]:
train_data = train[:6800000]
test_data = train[6800000:6803775]

In [32]:
import logging

import torch
import pandas as pd
from simpletransformers.ner import NERModel, NERArgs

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [33]:
model_args = NERArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.num_train_epochs = 10
model_args.overwrite_output_dir = True

In [ ]:
import torch
import pandas as pd
from simpletransformers.ner import NERModel, NERArgs

# Configure the model
ner_args = NERArgs()
ner_args.train_batch_size = 32
ner_args.evaluate_during_training = True
ner_args.overwrite_output_dir = True
ner_args.num_train_epochs = 7 #10

model = NERModel(
    "camembert", "airesearch/wangchanberta-base-att-spm-uncased", args=ner_args, use_cuda=torch.cuda.is_available(), labels=_CLAUSE_TAGS
)

# Train the model
model.train_model(train_data, eval_data=test_data)

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForTokenClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier

  0%|          | 0/6800000 [00:00<?, ?it/s]